# Function for Weighing Grid Cells by Land Area 
## Example Parameter: Avg TSA
#### Author: Sofia Ingersoll & Sujan Bhattaria
##### Date: 2024-01-15

##### Packages

In [1]:
# loading packages
import pandas as pd
import xarray as xr
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

##### Importing utils.py Functions 

In [2]:
from utils import *

##### Data
Both of these files will be stored in the utils library to aid in the automation of visual outputs.

In [4]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----     load data stored in casper     ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#-------Gridcell Landareas Data-----

# reading, storing, subsetting
landarea_file = '/glade/campaign/cgd/tss/projects/PPE/helpers/sparsegrid_landarea.nc'

landarea_ds = xr.open_dataset(landarea_file)

landarea = landarea_ds['landarea']


#-------Dummy Variable Data---------

# dummy data to have stored for preloaded visual on 
dummy_filepath = '/glade/campaign/cgd/tss/projects/PPE/PPEn11_OAAT/CTL2010/hist/PPEn11_CTL2010_OAAT0000.clm2.h0.2005-02-01-00000.nc'

# variable data described by filepath stored as ds
ds = xr.open_dataset(dummy_filepath)

# the desired variable may be subset from this data array
# da = ds[variable]

### Function to Automate Global Avg Weighted Landarea by Gridcell & Weighting Time

**Landarea**: Using the numpy function `weighted`, we can weigh the grid cell dimension of the data array according to landarea. 

**Time**: By grouping days in the month and year then averaging, an appropriate weight is applied to the time dimension.

The data used to define land area `landarea_file` will be preloaded by our system in the utils library, alongside the function below, upon final optimization.


***Note** The dummy variable data is currently labeled as ds, so this data can be plugged directly into the function below.* 

#### Building up function to include variable(s) selection

In [ ]:
# function to read in data using variable & time range of interest

# we'll need to consider what our User will be able to select on the Dashboard side of things.
# we want to create a drop down selection for variables and avaiable time ranges.
# this will aid in the data set scraping process.
# certain cut offs 
def ds_processing(variable, time_range):

#-------User Variable Data-----------------
# variable data described by filepath stored as ds
# reading, storing, subsetting
    
    # the structure for the end of this filepath string will need some more flexibility
    file = '/glade/campaign/cgd/tss/projects/PPE/PPEn11_OAAT/CTL2010/hist/PPEn11_CTL2010_OAAT0000.clm2.h0.2005-02-01-00000.nc'
    
    ds = xr.open_dataset(file)
    
    variable = ds[variable]



#------Weight Gridcells by Landarea---
    for gridcell in variable:
        variable.weighted(landarea)


#-----Calculating Variable Avg--------
variable_avg = variable.mean(dim = 'time')

#ds_processing(filepath_example, 'TSA')


#-----Generating Plot------------------
plt.figure(figsize = (9,4))
plt.scatter(ds.grid1d_lon,
            ds.grid1d_lat,
            c = variable_avg,
           cmap = 'RdPu',
           alpha = 0.5)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Default Simulation: OAAT0000')
plt.colorbar(label = 'Average TSA: 2m air temperature (K)')        # need to correct w/ f string. also want to implement f string for variable name in a few different places in this function

## Isolated functions that work properly

In [28]:
def yearly_weighted_average(da):
    # Get the array of number of days from the main dataset
    days_in_month = da['time.daysinmonth']
    # Multiply each month's data by corresponding days in month
    weighted_sum = (days_in_month*da).groupby("time.year").sum(dim = 'time') 
     # Total days in the year
    total_days = days_in_month.groupby("time.year").sum(dim = 'time') 
    
    return weighted_sum / total_days  

In [29]:
yearly_weighted_average(da)

<xarray.DataArray (year: 11, gridcell: 400)>
array([[278.40571019, 278.00984292, 279.5651003 , ..., 256.6019331 ,
        251.87584705, 260.12530275],
       [278.74014232, 278.49783016, 279.86800052, ..., 255.29325114,
        250.49749237, 258.45722585],
       [278.20645869, 278.14596123, 279.56326143, ..., 254.85145063,
        250.69051886, 258.18048539],
       ...,
       [277.89230673, 277.5988951 , 279.82024118, ..., 254.64756775,
        250.32953746, 257.78456011],
       [277.5026858 , 277.33597897, 279.49380894, ..., 255.26219574,
        250.5820356 , 258.49815093],
       [280.02102661, 280.7477417 , 281.85061646, ..., 244.4438324 ,
        238.31822205, 247.62683105]])
Coordinates:
  * year     (year) int64 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015
Dimensions without coordinates: gridcell

In [10]:
ds.TSA.dims

('time', 'gridcell')

In [11]:
file2 = '/glade/campaign/cgd/tss/projects/PPE/helpers/sparsegrid_landarea.nc'
ds2 = xr.open_dataset(file2)
landarea = ds2['landarea']
weighted_avg_area = ds.TSA.weighted(landarea).mean(dim = 'gridcell')
weighted_avg_area

<xarray.DataArray 'TSA' (time: 120)>
array([279.13522729, 279.83890245, 283.54835407, 287.55817442,
       290.87276366, 293.24786124, 294.19483278, 293.45801477,
       291.77527744, 288.28938363, 283.74954065, 280.11519891,
       278.31839777, 280.23906482, 283.30410677, 286.94442867,
       290.38120771, 293.13017602, 294.08733575, 293.51712022,
       291.52061172, 288.2042198 , 283.16067498, 280.601173  ,
       279.99635209, 280.32522872, 283.41555134, 287.71682648,
       290.73893482, 292.70378337, 293.99232631, 293.71023492,
       291.61800204, 288.06126085, 283.29573516, 279.79630537,
       278.22836951, 279.58458504, 283.97829794, 287.09264699,
       290.41382314, 292.79208376, 294.05428546, 293.46902976,
       291.2803967 , 288.20612216, 283.46345028, 279.77874934,
       278.96503224, 280.11570141, 283.15739974, 287.40704132,
       290.3874279 , 292.83141022, 293.89324054, 293.73541333,
       291.81709816, 288.03646217, 283.2745016 , 279.68249958,
       278.73275887, 280.39694658, 283.66812264, 287.62183069,
       290.99295759, 293.20748655, 294.45039144, 293.77821303,
       291.51579783, 288.03419692, 284.02119503, 279.5854227 ,
       278.59314757, 279.51182872, 282.92012283, 287.21017413,
       290.48158882, 293.0831642 , 294.25169691, 293.6789629 ,
       291.61470401, 288.20071994, 283.03836021, 280.1234707 ,
       278.23853014, 279.56811531, 282.87061355, 287.76772135,
       291.05956393, 293.27725876, 294.2565335 , 293.64250433,
       291.70200231, 288.17758253, 283.62441322, 279.29505021,
       278.79246809, 280.12211891, 283.0107463 , 286.87885651,
       290.7990445 , 293.13032978, 294.08605799, 293.49356481,
       291.505655  , 287.99675484, 283.84853166, 279.99637771,
       278.91499853, 279.43447807, 283.49570327, 287.61285238,
       290.8128853 , 293.19622162, 294.03771727, 293.72742055,
       291.57106709, 288.05586492, 283.15399448, 280.26512635])
Coordinates:
  * time     (time) object 2005-02-01 00:00:00 ... 2015-01-01 00:00:00

In [12]:
(1/landarea.sum())*(landarea*ds.TSA).sum(dim = 'gridcell')

<xarray.DataArray (time: 120)>
array([279.13522729, 279.83890245, 283.54835407, 287.55817442,
       290.87276366, 293.24786124, 294.19483278, 293.45801477,
       291.77527744, 288.28938363, 283.74954065, 280.11519891,
       278.31839777, 280.23906482, 283.30410677, 286.94442867,
       290.38120771, 293.13017602, 294.08733575, 293.51712022,
       291.52061172, 288.2042198 , 283.16067498, 280.601173  ,
       279.99635209, 280.32522872, 283.41555134, 287.71682648,
       290.73893482, 292.70378337, 293.99232631, 293.71023492,
       291.61800204, 288.06126085, 283.29573516, 279.79630537,
       278.22836951, 279.58458504, 283.97829794, 287.09264699,
       290.41382314, 292.79208376, 294.05428546, 293.46902976,
       291.2803967 , 288.20612216, 283.46345028, 279.77874934,
       278.96503224, 280.11570141, 283.15739974, 287.40704132,
       290.3874279 , 292.83141022, 293.89324054, 293.73541333,
       291.81709816, 288.03646217, 283.2745016 , 279.68249958,
       278.73275887, 280.39694658, 283.66812264, 287.62183069,
       290.99295759, 293.20748655, 294.45039144, 293.77821303,
       291.51579783, 288.03419692, 284.02119503, 279.5854227 ,
       278.59314757, 279.51182872, 282.92012283, 287.21017413,
       290.48158882, 293.0831642 , 294.25169691, 293.6789629 ,
       291.61470401, 288.20071994, 283.03836021, 280.1234707 ,
       278.23853014, 279.56811531, 282.87061355, 287.76772135,
       291.05956393, 293.27725876, 294.2565335 , 293.64250433,
       291.70200231, 288.17758253, 283.62441322, 279.29505021,
       278.79246809, 280.12211891, 283.0107463 , 286.87885651,
       290.7990445 , 293.13032978, 294.08605799, 293.49356481,
       291.505655  , 287.99675484, 283.84853166, 279.99637771,
       278.91499853, 279.43447807, 283.49570327, 287.61285238,
       290.8128853 , 293.19622162, 294.03771727, 293.72742055,
       291.57106709, 288.05586492, 283.15399448, 280.26512635])
Coordinates:
  * time     (time) object 2005-02-01 00:00:00 ... 2015-01-01 00:00:00